In [ ]:
!pip install flask_ngrok
!pip install pororo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.8/748.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, current_app , redirect, url_for
# from flask_restful import Resource, Api
import cv2
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pororo import Pororo
from imutils.perspective import four_point_transform
from imutils.contours import sort_contours
import imutils
import re

In [ ]:
# pip install opencv-contrib-python

In [ ]:
# ocr = Pororo(task="ocr", lang="ko")

# str_list = ocr(img_path)
# result = listToString(str_list)
# ner = Pororo(task="ner", lang="ko")
# ner(result)
# # ocr(img_path)

In [ ]:
def plt_imshow(title='image',img=None, figsize=(8,5)):
    plt.figure(figsize=figsize)
    
    if type(img) == list :
        if type(title) == list:
            titles = title
        else:
            titles = []

            for i in range(len(img)):
                titles.append(title)

        for i in range(len(img)):
            if len(img[i].shape) <=2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else :
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)

            plt.subplot(1,len(img), i +1),plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
        plt.show()
    
    else :
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        else :
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

In [ ]:
def make_scan_image(image,width,ksize=(5,5),min_threshold=75,max_threshold=200):
    image_list_title = []
    image_list = []
    
    org_image = image.copy()
    image = imutils.resize(image, width=width)
    ratio = org_image.shape[1] / float(image.shape[1])
    
    # 이미지를 grayscale로 변환하고 blur를 적용
    # 모서리르 ㄹ 찾기위한 이미지 연산
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, ksize, 1)
   
    edged = cv2.Canny(blurred, min_threshold, max_threshold)
    
    image_list_title = ['gray', 'blurred', 'edged']
    image_list = [gray, blurred, edged]
    
    # contours를 찾아 크기순으로 정렬
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    findCnt = None
    
    # 정렬된 contours를 반복문으로 수행하며 4개의 꼭지점을 갖는 도형을 검출
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02*peri, True)
        
        # contours가 크기순으로 정렬되어 있기 때문에 제일 첫번째 사각형을 영역으로 판단하고 break
        if len(approx) ==4:
            findCnt = approx
            break
        
    
    if findCnt is None :
        raise Exception(("Could not find out line"))
    output = image.copy()
    cv2.drawContours(output, [findCnt], -1, (0,255,0),2)
    
    image_list_title.append("Outline")
    image_list.append(output)
    
    # 원본 이미지에 찾은 윤곽을 기준으로 이미지를 보정
    transform_image = four_point_transform(org_image, findCnt.reshape(4,2)*ratio)
    
    img = sr.upsample(transform_image,4)
    plt_imshow(image_list_title, image_list)
    plt_imshow("Transform",transform_image )
    
    return transform_image
    

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()




In [ ]:
ocr = Pororo(task="ocr", lang="ko")
ner = Pororo(task="ner", lang="ko")

In [ ]:
app = Flask(__name__)
run_with_ngrok(app) 

data_list2 = []
@app.route('/')
def index(methods=['GET', 'POST']):
    if request.method == 'GET':
        return 'GET'
    else:
        return 'POST'


@app.route('/test/',methods=['GET','POST'])
def testJson():
    if request.method == 'GET':
        print("겟포스트")
        return 'GET'
    
    else:
        print(request.files['test'])
        
        f = request.files['test']
#         img2 = Image.open(f)
#         img2 = img2.convert('L') 
        
#         print(f.read())
#         cv_file = cv2.FileStorage(f, cv2.FILE_STORAGE_READ)
        f.save(f.filename)
        img2 = cv2.imread(f.filename, cv2.IMREAD_COLOR)
        
        print(img2.shape,'이미지 모양')
        # 
        img = cv2.resize(img2, (1920, 1080),interpolation=cv2.INTER_LANCZOS4)
        # img = sr.upsample(img,4)
        # imgread = make_scan_image(img2,width=100, ksize=(5,5), min_threshold=50, max_threshold=100)
        plt_imshow("이미지",img)
        
        str_list = ocr(img)
        
        result = listToString(str_list)
        
        data_list = ner(result)
        data_list2 = data_list
        print(data_list)

        #add
        new_list =[]
        for x in ner(result): 
          if "PERSON" in x:
            new_list.append(x)
            
        pattern = '[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
        pattern2 = '[01|10]\d{1,2}[.]\d{3,4}[.]\d{4}|[01|10]\d{1,2}.\d{3,4}.\d{4}|[01|10]\d{1,2}-\d{3,4}-\d{4}'
        # regex = re.compile(pattern, flags=re.IGNORECASE)
        regex2 = re.compile(r'(\d{2|3})-(\d{4}-\d{4})')
        regex3 = re.compile(r'(\d{2|3})-(\d{3}-\d{4})|((\d{2|3}).\d{2|3}).(\d{3}.\d{4})')
        regex = re.compile(pattern, flags=re.IGNORECASE)

        phonenum = ""
        email = ""
        name = ""
        soft = ""
        grade = ""
        fax = ""
        office = ""
        check = 0
        for b in data_list2:
          if b[1] != "O":
            if b[1] == "QUANTITY":
              if len(b[0]) > 10 :
                print(b)
                print(regex2.match(b[0]))
                if check == 0 :
                  phonenum = b[0]
                  check +=1
                elif check == 1:
                  fax = b[0]
                  check +=1
                elif check == 2:
                  office = b[0]
            elif b[1] == "TERM":
              print(b)
              print(regex.match(b[0]))
              if regex.match(b[0]) != None :
                email = b[0]              
            elif b[1] == "PERSON" and name == "":
              print(b)
              nameto = b[0]
              name = nameto.replace(" ","")
            elif b[1] == "ORGANIZATION" and soft == "":
              soft = b[0]
            elif b[1] == "OCCUPATION" :
              grade = b[0]
        check == 0
        return jsonify({'names': name,
                        'grade': grade,
                        "phnum": phonenum,
                       "email": email,
                       "office":soft,
                      "officenum":office,
                      "fax" : fax
                        })
    
    


app.run()

In [ ]:
sr = cv2.dnn_superres.DnnSuperResImpl_create()
 
path = "EDSR_x4.pb"
 
sr.readModel(path)
sr.setModel("edsr", 4)
 

In [ ]:
img = cv2.imread("20230215_111130.jpg")

img = sr.upsample(img,4)

plt_imshow("test",img)

In [ ]:
data_list2 = [('정 유 진', 'PERSON'), (' ', 'O'), ('연구원', 'OCCUPATION'), (' ', 'O'), ('YuJin Jeong', 'PERSON'), (' ', 'O'), ('Researche', 'CIVILIZATION'), ('r', 'O'), (' ', 'O'), ('(주)에스엠소프트', 'ORGANIZATION'), (' ', 'O'), ('에너지', 'O'), (' ', 'O'), ('조선', 'O'), (' ', 'O'), ('ICT', 'STUDY_FIELD'), (' ', 'O'), ('전자부품', 'O'), (' ', 'O'), ('에스엠소프트', 'ORGANIZATION'), (' ', 'O'), ('드론', 'TERM'), ('제조', 'O'), (' ', 'O'), ('IoT', 'STUDY_FIELD'), (' ', 'O'), ('AI', 'STUDY_FIELD'), (' ', 'O'), ('B', 'STUDY_FIELD'), ('igDat', 'STUDY_FIELD'), ('a', 'O'), (' ', 'O'), ('A', 'STUDY_FIELD'), ('R', 'O'), (' ', 'O'), ('ADORESS', 'O'), (' ', 'O'), ('전라남도', 'LOCATION'), (' ', 'O'), ('나주시', 'LOCATION'), (' ', 'O'), ('도민길', 'ARTIFACT'), (' ', 'O'), ('50', 'QUANTITY'), (',(', 'O'), ('빛가람동', 'ARTIFACT'), (')', 'O'), (' ', 'O'), ('58S23', 'QUANTITY'), (' ', 'O'), ('50', 'QUANTITY'), (',', 'O'), (' ', 'O'), ('Domin-gil', 'ARTIFACT'), (',', 'O'), (' ', 'O'), ('Naju-si', 'ARTIFACT'), (',', 'O'), (' ', 'O'), ('jeollanam-do', 'ARTIFACT'), ('.', 'O'), (' ', 'O'), ('Korea', 'LOCATION'), (' ', 'O'), ('INNO-BIZ', 'ARTIFACT'), (' ', 'O'), ('기업', 'O'), (' ', 'O'), ('MAIN 81z', 'QUANTITY'), (' ', 'O'), ('기업', 'O'), (' ', 'O'), ('전남국방벤처센터', 'ORGANIZATION'), (' ', 'O'), ('협약기업', 'O'), (' ', 'O'), ('한국전력', 'ORGANIZATION'), (' ', 'O'), ('스타기업', 'O'), (' ', 'O'), ('전라남도', 'LOCATION'), (' ', 'O'), ('스타기업', 'O'), (' ', 'O'), ('전라남도', 'LOCATION'), (' ', 'O'), ('유망중소기업', 'O'), (' ', 'O'), ('청년친화', 'O'), (' ', 'O'), ('강소기업', 'O'), (' ', 'O'), ('TEL', 'O'), (' ', 'O'), ('+82.61.333.8517', 'QUANTITY'), (' ', 'O'), ('FAX', 'O'), (' ', 'O'), ('+82.61.333.8516', 'QUANTITY'), (' ', 'O'), ('MOBILE', 'O'), (' ', 'O'), ('+82.10.4234.8981', 'QUANTITY'), (' ', 'O'), ('E-MAlL', 'O'), (' ', 'O'), ('yj.jeong@smsoft.co.kr', 'TERM'), (' ', 'O'), ('HOMEPAGE', 'O'), (' ', 'O'), ('www.smsoft.co.kr', 'TERM')]

In [ ]:
for b in data_list2:
  if b[1] != "O":
    if b[1] == "PERSON":
      print(b)

In [ ]:
for b in data_list2:
  if b[1] != "O":
    if b[1] == "TERM":
      print(b)

In [ ]:
for b in data_list2:
  if b[1] != "O":
    if b[1] == "OCCUPATION":
      print(b)


In [ ]:
for b in data_list2:
  if b[1] != "O":
    if b[1] == "ORGANIZATION":
      print(b)



In [ ]:
for b in data_list2:
  if b[1] != "O":
    if b[1] == "QUANTITY":
      print(b)


In [ ]:

print(name)
print(phonenum)
print(grade)
print(soft)
print(email)
